# API Gateway Demo

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from client_utils import (
    populate_bucket,
    get_session_token,
    get_job_progress,
    get_job_results,
    get_overall_progress,
    update_job_results,
    create_dummy_job,
)

uploads_bucket_name = "ai-description-dev-nt-isngd-2-381491992967-uploads"
cloudfront_distribution = "d1d2rv3wk2im6q.cloudfront.net"
api_url = f"https://{cloudfront_distribution}/api/"
# api_url = "https://hpq6dsoez6.execute-api.us-east-1.amazonaws.com/dev/"
job_name = "job_001"
bias_job_name = f"{job_name}_bias"
metadata_job_name = f"{job_name}_metadata"

## Populate bucket

In [ ]:
image_s3_uri, original_metadata_s3_uri, context_s3_uri = populate_bucket(
    bucket_name=uploads_bucket_name,
    image_fpath="../tests/data/offensive_image.jpg",
)

## Get session token

In [ ]:
session_token = get_session_token(
    api_url=api_url,
    username="guest",
    password="password",
)

In [ ]:
create_dummy_job(
    api_url=api_url,
    job_name=bias_job_name,
    job_type="bias",
    original_metadata_s3_uri=original_metadata_s3_uri,
    context_s3_uri=context_s3_uri,
    image_s3_uri=image_s3_uri,
    session_token=session_token,
)

create_dummy_job(
    api_url=api_url,
    job_name=metadata_job_name,
    job_type="metadata",
    original_metadata_s3_uri=original_metadata_s3_uri,
    context_s3_uri=context_s3_uri,
    image_s3_uri=image_s3_uri,
    session_token=session_token,
)

## Check Job Progress

In [ ]:
bias_progress = get_job_progress(
    api_url=api_url,
    job_name=bias_job_name,
    session_token=session_token,
)
for key, val in bias_progress.items():
    print(f"{key}: {val}")

metadata_progress = get_job_progress(
    api_url=api_url,
    job_name=metadata_job_name,
    session_token=session_token,
)
for key, val in metadata_progress.items():
    print(f"{key}: {val}")

## Get overall progress

In [ ]:
overall_progress = get_overall_progress(
    api_url=api_url,
    session_token=session_token,
)
for key, val in overall_progress.items():
    print(f"{key}: {val}")

## Get Job Results

In [ ]:
item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][0],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][1],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=metadata_job_name,
    work_id=metadata_progress["job_progress"]["READY FOR REVIEW"][0],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=metadata_job_name,
    work_id=metadata_progress["job_progress"]["READY FOR REVIEW"][1],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")

## Update Job Results

In [ ]:
update_job_results(
    api_url=api_url,
    job_name=job_name,
    work_id=bias_progress["READY_FOR_REVIEW"][0],
    session_token=session_token,
)